## Area Level Cyclability

### Structure

- Get LSOAs / local authority boundaries
- For a single LSOA
    - Get historical data (2016 vs 2021)
    - Some quality checks on data
    - Calculate LTS 2016 vs 2021
    - Analyse impact over time
- Network measures using LTS
    - Connectivity at different LTS levels
    - Other graph measures, separate research to bring in idea/insights for this section

In [1]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

In [15]:
ox.settings.overpass_settings = '[out:json][timeout:90][date:"2016-10-28T19:20:00Z"]'

In [2]:
#Import LSOAs

lsoas = gpd.read_file('data/LSOA_2011_Boundaries_Super_Generalised_Clipped_BSC_EW_V4_6029841263726194941.gpkg').to_crs(4326)
lsoas = pd.concat([lsoas, lsoas.bounds], axis=1)

In [3]:
lsoa = lsoas.iloc[0]

In [16]:
G_2019 = ox.graph_from_bbox(lsoa['maxy'],lsoa['miny'],lsoa['minx'],lsoa['maxx'],network_type = 'bike',retain_all=True,simplify=False)

In [17]:
edge_attributes = ox.graph_to_gdfs(G_2019, nodes=True)[1]

In [19]:
edge_attributes_gdf = gpd.GeoDataFrame(edge_attributes[['osmid','geometry']], crs="EPSG:4326", geometry=edge_attributes['geometry'])
edge_attributes_gdf.to_file('data/outputs/lsoa_example.geojson', driver='GeoJSON')